<a href="https://colab.research.google.com/github/DavisRayM/msft-stock-prediction/blob/main/msft-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4

Author: Davis Muro

For CPSC 5610

Microsoft Stock Prediction

In [29]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
df = pd.read_csv('data/MSFT.csv')
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,11/7/2016,59.779999,60.520000,59.779999,60.419998,55.902321,31664800
1,11/8/2016,60.549999,60.779999,60.150002,60.470001,55.948589,22935400
2,11/9/2016,60.000000,60.590000,59.200001,60.169998,55.671009,49632500
3,11/10/2016,60.480000,60.490002,57.630001,58.700001,54.310928,57822400
4,11/11/2016,58.230000,59.119999,58.009998,59.020000,54.607002,38767800


## Preprocessing

In [31]:
# Convert `Date` to `DateTime`
df['Date'] = pd.to_datetime(df['Date'])

# Sort dataframe by `Date` (Ascending)
df.sort_values(by='Date', inplace=True, ascending=True)

# Drop Adj Close
df.drop(columns=['Adj Close'], inplace=True)

df.head(5)

,Date,Open,High,Low,Close,Volume
0,2016-11-07,59.779999,60.520000,59.779999,60.419998,31664800
1,2016-11-08,60.549999,60.779999,60.150002,60.470001,22935400
2,2016-11-09,60.000000,60.590000,59.200001,60.169998,49632500
3,2016-11-10,60.480000,60.490002,57.630001,58.700001,57822400
4,2016-11-11,58.230000,59.119999,58.009998,59.020000,38767800


In [33]:
from sklearn.preprocessing import MinMaxScaler

# Normalize numerical columns using MinMaxScaler
scaler = MinMaxScaler()
df[['Close', 'High', 'Low', 'Open', 'Volume']] = scaler.fit_transform(df[['Close', 'High', 'Low', 'Open', 'Volume']])

df.head(5)

,Date,Open,High,Low,Close,Volume
0,2016-11-07,0.005530,0.005148,0.009021,0.008264,0.233481
1,2016-11-08,0.008277,0.006078,0.010356,0.008444,0.149396
2,2016-11-09,0.006315,0.005398,0.006928,0.007366,0.406553
3,2016-11-10,0.008028,0.005041,0.001263,0.002084,0.485441
4,2016-11-11,0.000000,0.000143,0.002634,0.003234,0.301900


In [41]:
# Set `Date` as index; It's unique
if (df.duplicated(subset=['Date']).sum() > 0):
    assert False, "Duplicate dates found"

df.set_index('Date', inplace=True)
df.head(5)

,Open,High,Low,Close,Volume
Date,,,,,
2016-11-07,0.005530,0.005148,0.009021,0.008264,0.233481
2016-11-08,0.008277,0.006078,0.010356,0.008444,0.149396
2016-11-09,0.006315,0.005398,0.006928,0.007366,0.406553
2016-11-10,0.008028,0.005041,0.001263,0.002084,0.485441
2016-11-11,0.000000,0.000143,0.002634,0.003234,0.301900


## Windowing